# Ticket Return to Hubspot ETL

This code describes the process to transfer ticket transcation data from Quickbase to Hubspot. This data supports ROI tracking on marketing assets and customer segmentation for sales/marketing strategies.


### Step 1 - Extract data from Quickbase

Ticketing data needs to be exported from the "Tickets" table in Quickbase.

When done as part of a daily ETL process, data can be downloaded by exporting this table in Quickbase: <a href="https://alexanderbrown.quickbase.com/nav/app/btkpb7gci/table/btkpff4h3/action/q?qid=1000018&skip=0">Tickets default report</a> <br>
Customer data will also need to be exported_dealsed with this table in Quickbase, saved as 'customers.csv', this will be used in Step 3: <a href="https://alexanderbrown.quickbase.com/nav/app/btkpb7gci/table/btkpgag95/action/q?qid=1000005&skip=0">Customers default report</a>

However, Quickbase has a limitation in how much data can be exported_dealsed at once. For large exported_dealss representing 30+ days, we need to exported_deals the data in batches as seperate files. To extract all 2024 data, we filter the <link> table on two dimensions to get smaller datasets.
* Bowie tickets associated with a customer email address
* Bowie tickets not associated with a customer email address
* Frederick tickets associated with a customer email address
* Frederick tickets not associated with a customer email address


The downloaded files should be stored in the same local directory as this script.

Columns are then mapped from the quickbase columns to the to HubSpot naming convetions

<b> Lastly, we add the appropriate season to the property. </b> When doing future updates, it is key to modify this field to be attributable of the current season.

In [1]:
import pandas as pd

# Read each CSV into a separate dataframe
bowie_email = pd.read_csv("Baysox 2024 Email.csv")
bowie_no_email = pd.read_csv("Baysox 2024 no Email.csv")
keys_email = pd.read_csv("Keys 2024 Email.csv")
keys_no_email = pd.read_csv("Keys 2024 no Email.csv")

# Concatenate all dataframes into a single one
tickets = pd.concat([bowie_email, bowie_no_email, keys_email, keys_no_email], ignore_index=True)

# Dictionary to map old column names to new column names
column_mapping = {
    "Customer ID": "Customer ID",
    "Customer ID - Email": "Email",
    "Event ID - Sponsor - Team": "Business unit",
    "Event ID - Event Date": "Event date",
    "IsScanned": "IsScanned",
    "Section": "Section",
    "Ticket product - Ticket Type": "Ticket Type",
    "TicketPrice": "TicketPrice",
    "TransactionID": "TransactionID",
    "Transaction Date": "Close Date",
    "Event ID - Sponsor - Year": "Season"
}

# Rename the columns in the dataframe
tickets = tickets.rename(columns=column_mapping)

# Add Season column used to distinguish the season these tickets are from
Season = 2024

tickets.head()


,Customer ID,Email,Business unit,Event date,IsScanned,Section,Ticket Type,TicketPrice,TransactionID,Close Date,Season
0,1220577.0,redgoing@hotmail.com,Baysox,08-09-2024,no,104,Group Ticket,$18,478864,08-02-2024 12:51 PM,2024
1,1220577.0,redgoing@hotmail.com,Baysox,08-09-2024,no,104,Group Ticket,$18,478864,08-02-2024 12:51 PM,2024
2,1220577.0,redgoing@hotmail.com,Baysox,08-09-2024,no,104,Group Ticket,$18,478864,08-02-2024 12:51 PM,2024
3,1220577.0,redgoing@hotmail.com,Baysox,08-09-2024,no,104,Group Ticket,$18,478864,08-02-2024 12:51 PM,2024
4,1220577.0,redgoing@hotmail.com,Baysox,08-09-2024,no,104,Group Ticket,$18,478864,08-02-2024 12:51 PM,2024


### Step 2 - Deal Data Transformation

We perform several data transformations to improve data quality and make the data acceptable for import to Hubspot

1. <b> Email domain corrections </b> -- Replace invalid domain exentions like ".comm" with ".com" or their likely equaivalent
2. <b> Define a new transcation id </b> -- Ticket Return's [Transaction ID] values are only unique to the organization (Bowie or Frederick). We set [Transaction ID] = "[Organization]-[Transaction ID]"
3. <b> Convert tickets scanned to 1,0 </b> -- Reconfigure [IsScanned] to binary to sum when grouping to get the total number of tickets scanned per transaction
4. <b> Set ticket price to numeric </b> -- Originally had issues with some prices being in different formats, this ensures there are no issues when calculating total price
5. <b> Fill null types to Unknown </b> -- Mark tickets without a type as unknown to distinguish them
6. <b> Set price for certain ticket types to 0 </b> -- Certain deals are already tracked in HubSpot, we set these types to zero as to not double count our revenues
7. <b> Adjusting close date </b> -- Some transactions have tickets without close dates, these are populated with the same date as the rest of the transaction or the event date as when importing to HubSpot if there is no closed date on a closed won deal it will default to the create date and we don't want deals to have close dates after the events for individual tickets

Following the transformations we store our dataframe in a new one called line_items to use later to import the line items connecting to the deals.


In [2]:
import warnings

# Function to filter out malformed emails (doesn't contain @ symbol)
def format_emails(email):
    if pd.notna(email) and '@' in email:
        return email
    return ''

# Function to fix invalid emails
def format_invalid_emails(email):
    com_variations = [',com', '.cim', '.ocm', '.C0M', '.comm', '. COM', '.coom', '.cop', '.oom', '.lcom', '.con', '.co,m', '..com', '.cpm', '.cpom', '.col', '.co,', '.xom', '.fom', '.coke', 'cok', '.come', '..com']
    net_variations = [',net', '.met', '.ent', '.nat', '.nt', '.ner', '.ney', '.lnet', '. net', '.nety', '.neto', '.ncet', '/net', '.nedt', '.nett', '.nbet']
    
    # Skip processing if the email is NaN or empty
    if pd.isna(email) or email == '':
        return email
    
    # Remove spaces from the email
    email = email.replace(" ", "")
    
    # Fix .com variations
    for variation in com_variations:
        if variation in email:
            email = email.replace(variation, ".com")
            break
    
    # Fix .net variations
    for variation in net_variations:
        if variation in email:
            email = email.replace(variation, ".net")
            break

    # Remove consecutive dots
    while '..' in email:
        email = email.replace('..', '.')
    
    return email

# Apply the functions, skipping NaN values
tickets['Email'] = tickets['Email'].apply(lambda x: format_emails(x) if pd.notna(x) else x)
tickets['Email'] = tickets['Email'].apply(lambda x: format_invalid_emails(x) if pd.notna(x) else x)

#tickets.to_csv('test.csv')
tickets['TransactionID'] = tickets['Business unit'] + '-' + tickets['TransactionID'].astype(str)

# 1. Change 'IsScanned' from 'yes/no' to 1/0
tickets['IsScanned'] = tickets['IsScanned'].apply(lambda x: 1 if x.lower() == 'yes' else 0)

# Remove any dollar signs and commas from 'TicketPrice' and convert to float
tickets['TicketPrice'] = tickets['TicketPrice'].replace('[\$,]', '', regex=True).astype(float)

# Replace NaN values in 'Ticket Type' with 'Unknown'
tickets['Ticket Type'] = tickets['Ticket Type'].fillna('Unknown')

# Define the ticket types that should have PriceCorrected set to 0
invalid_ticket_types = ['Group Ticket', 'Ticket Plan', 'Ticket Package', 'Unknown', 'Non-Ticket Inventory']

# Create a new column 'PriceCorrected' based on the condition
tickets['PriceCorrected'] = tickets.apply(lambda row: 0 if row['Ticket Type'] in invalid_ticket_types else row['TicketPrice'], axis=1)

# Convert 'Close Date' to datetime with time
tickets['Close Date'] = pd.to_datetime(tickets['Close Date'], errors='coerce')

# Convert 'Event date' to datetime (without time)
tickets['Event date'] = pd.to_datetime(tickets['Event date'], errors='coerce').dt.date

# Check for NULL values in 'Close Date' and issue a warning if any are found
if tickets['Close Date'].isna().any():
    warnings.warn("NULL values detected in 'Close Date'. This should not occur as per Quickbase requirements. Please check upstream data feed from TR to Quickbase.")

# Continue with storing missing 'Close Date' rows for reference
missing_closed_dates = tickets[tickets['Close Date'].isna()].copy()
missing_closed_dates.drop_duplicates(subset=['TransactionID']).to_csv('missing_closed_dates.csv')


# Fill missing 'Close Date' values using 'Event date' with a default time of 9 AM
tickets['Close Date'] = tickets['Close Date'].fillna(
    pd.to_datetime(tickets['Event date'].astype(str) + ' 09:00')
)

#Save line items for later use after we group by for transactions
line_items = tickets

tickets.to_csv('line items.csv')



C:\Users\MaxStevens\AppData\Local\Temp\ipykernel_16836\4285153792.py:69: UserWarning: NULL values detected in 'Close Date'. This should not occur as per Quickbase requirements. Please check upstream data feed from TR to Quickbase.
  warnings.warn("NULL values detected in 'Close Date'. This should not occur as per Quickbase requirements. Please check upstream data feed from TR to Quickbase.")


### Step 3 - Import New Contacts to Hubspot

We must first migrate our unique contacts to HubSpot to ensure we don't have errors upon importing deals, there are two types of errors we must account for:

1. <b> Invalid Email </b> -- This we solve above when correcting the email domains, HubSpot will only import valid email addresses
2. <b> Invalid Duplicate ID </b> -- This error occurs when a contact is created in an import then appears again in the import. Since we are importing multiple deals associating to contacts we are at risk of this error. This only occurs when a contact is created in the import so by first importing the contacts then the deals we ensure that we won't create any contacts when importing deals removing this risk

When importing these contacts we want to ensure we are including all the data exported from QuickBase into HubSpot. We also want to ensure this data is clean and that contacts are being put in the correct business unit. The exported file from Step 1 has the correct columns needed. This code below will clean and match them to get us an export of unique emails with cleaned information

1. <b> Find Business units </b> -- Our customers are exported with the # of tickets they purchased for each team. We use this to find if they belong in a unique business unit or both
2. <b> Add Contact owner </b> -- Based on the business units the respective contact owners are added
3. <b> Clean customer emails </b> -- Since we are matching on emails to those pulled from our tickets, we must ensure they are in the same format as them. To do this we run them through the same email cleaning process then set both emails to be lowercase in order to help with matching and uniqueness 
4. <b> Clean customer columns </b> -- We then want to clean the customers data that we will be importing into HubSpot. This includes formating name, address, and phone number data as well as removing the columns no longer needed
5. <b> Merge and drop duplicates </b> -- Last step is to merge and drop the duplicates so we don't import the same email twice which will cause an error

After these steps are followed you are left with a file to import into HubSpot which will ensure all of our customers contacts are in the system before importing the deals which will allow them to match. When importing, match each column to its corresponding HubSpot column, all of these are self explanatory when matching. It is important that you <b> do not overwrite </b> is selected for all columns

##### <b> Do not overwrite columns on contact import </b>

In [3]:
import re

# 3. Remove NaN and empty emails, ensure uniqueness
unique_emails = tickets['Email'].dropna().replace('', pd.NA).dropna().str.strip().unique()

# Create a new DataFrame with unique emails
tickets_unique_emails = pd.DataFrame(unique_emails, columns=['Email'])

# Import the customers into a DataFrame
customers = pd.read_csv('customers.csv')

# Function to format phone numbers
def format_phone_number(phone):
    if pd.isna(phone) or isinstance(phone, str) and not re.search(r'\d', phone):
        return None
    else:
        clean_phone = re.sub(r'[^\d]', '', str(phone))  # Remove all non-digit characters
        if re.match(r'(\d{10})$', clean_phone):
            # category 1, complete number
            formatted_number = f'+1{clean_phone}'
            return formatted_number
        else:
            return None

# Function to format zip codes
def format_zip_codes(zip_code):
    if pd.isna(zip_code) or not isinstance(zip_code, str):
        return ''
    zip_code = re.sub(r'\D', '', zip_code)  # Remove non-digit characters
    if len(zip_code) >= 5:
        return zip_code[:5]  # Return the first 5 characters
    return ''

# Function to determine the business unit
def determine_business_unit(row):
    bowie_tickets = int(row['# of Bowie Tickets'])
    frederick_tickets = int(row['# of Frederick Tickets'])
    business_units = []
    
    if bowie_tickets > 0:
        business_units.append('Bowie Baysox')
    if frederick_tickets > 0:
        business_units.append('Frederick Keys')
    
    return ';'.join([''] + business_units) if business_units else None

# Function to determine the contact owner for Bowie Baysox
def contact_owner_bowie(business_unit):
    if business_unit and 'Bowie Baysox' in business_unit:
        return 'info@baysox.com'
    return None

# Function to determine the contact owner for Frederick Keys
def contact_owner_frederick(business_unit):
    if business_unit and 'Frederick Keys' in business_unit:
        return 'info@frederickkeys.com'
    return None

# Create the 'Business Unit' column
customers['Business Unit'] = customers.apply(determine_business_unit, axis=1)

# Create the 'Contact Owner - Bowie Baysox' column
customers['Contact Owner - Bowie Baysox'] = customers['Business Unit'].apply(contact_owner_bowie)

# Create the 'Contact Owner - Frederick Keys' column
customers['Contact Owner - Frederick Keys'] = customers['Business Unit'].apply(contact_owner_frederick)

# Apply the functions to clean 'Email', skipping NaN values
customers['Email'] = customers['Email'].apply(lambda x: format_emails(x) if pd.notna(x) else x)
customers['Email'] = customers['Email'].apply(lambda x: format_invalid_emails(x) if pd.notna(x) else x)

# Convert email addresses to lowercase in both dataframes to allow for consistent matching and uniqueness
customers['Email'] = customers['Email'].str.lower()
tickets_unique_emails['Email'] = tickets_unique_emails['Email'].str.lower()

# Set country to US (comes as USA)
customers['Country'] = 'US'

# Ensure columns are in title case
customers['First Name'] = customers['First Name'].str.title()
customers['Last Name'] = customers['Last Name'].str.title()
customers['Addr1'] = customers['Addr1'].str.title()
customers['Addr2'] = customers['Addr2'].str.title()
customers['City'] = customers['City'].str.title()

# Apply the function to the 'Phone' column to clean
customers['Phone'] = customers['Phone'].apply(format_phone_number)

# Apply the function to the 'Zip' column to clean
customers['Zip'] = customers['Zip'].apply(format_zip_codes)

# Drop specified columns
customers = customers.drop(columns=['# of Bowie Tickets', '# of Frederick Tickets'])

# Rename columns in the customers dataframe
customers.rename(columns={
    'Addr1': 'Street Address',
    'Addr2': 'Street Address Line 2',
    'Zip': 'Postal Code'
}, inplace=True)

# Merge on 'Email' column
cleaned_emails = pd.merge(customers, tickets_unique_emails, on='Email', how='inner')

# Drop duplicate emails if any exist after merging
cleaned_emails = cleaned_emails.drop_duplicates(subset='Email')

# Save file for Import
cleaned_emails.to_csv('2024.11.11 - Attain - Individual Tickets Customer Update.csv')

## Step 4 - Transform Tickets into Transcations for Import

Now we have all our tickets cleaned up we are ready to group these into their specific transactions which will be the deals we will import

1. <b> Sort by Email </b> -- For some tickets in QuickBase the email did not populate all instances, this ensure we will have an email value if there is one associated with any of the tickets in the transaction
2. <b> Define prioritization of tickets </b> -- We want to prioritize what ticket type will assume the primary name of the [TicketType] in the Deal. When we import our Deals into HubSpot we can only have one TicketType and since we are counting revenue based on Individual, Exchange, and Comp only those will be our priority. When we import the line items it will include the further breakdown into each individual type
3. <b> Group into transactions </b> -- Grouping is done by [TransactionID] and [Event Date] so we can see the specific breakdown of games attended. After grouping the required HubSpot properties are added and we set the [Deal Owner] and [BusinessUnit] to the correct name. These Deals are then named and ready for import into HubSpot.

We then split the DataFrame into two, one with known email and one with unknown email <br>

The reason for doing this is when importing into HubSpot we want to do two different imports as importing some with emails and some without will cause errors. The way to do each of these imports is listed below:
1. <b> Email</b>: Select Deals and Contacts on import, do create deals only and update contacts only
2. <b> No Email</b>: Select Deals on import, do create deals only

All column names match those they are importing to but one area to keep a note on is when importing the [Close Date] property will default to be a contact property. <b> Close Date must be switched to the deal property under the same name or the deal closed dates will be set to the import date. </b>
    

In [ ]:
tickets = tickets.sort_values(by=['Email', 'TransactionID'], ascending=[True, True])

# Define a function to prioritize TicketType
def prioritize_ticket_type(ticket_types):
    # Define the priority order
    priority_order = ['Individual Ticket', 'Exchange Ticket', 'Comp Ticket', 'Ticket Plan', 'Group Ticket', 'Ticket Package']
    
    # Remove NaN values and sort based on the priority
    ticket_types = [t for t in ticket_types if pd.notna(t)]
    for priority in priority_order:
        if priority in ticket_types:
            return priority
    return 'Other'  # Default if none of the types match

# Group by TransactionID and Event Date
deals = tickets.groupby(['TransactionID', 'Event date']).agg(
    TotalTickets=('TransactionID', 'size'),            # Count the number of rows
    TotalPrice=('PriceCorrected', 'sum'),              # Sum of PriceCorrected
    TicketType=('Ticket Type', prioritize_ticket_type),               # First instance of Ticket Type
    Section=('Section', 'first'),                      # First instance of Section
    BusinessUnit=('Business unit', 'first'),           # First instance of Business Unit
    NumberOfTicketsScanned=('IsScanned', 'sum'),       # Sum of IsScanned
    CloseDate=('Close Date', 'first'),                 # First instance of Close Date
    Email=('Email', 'first')                           # First instance of Email
).reset_index()

# Add the new columns
deals['Pipeline'] = 'Individual Tickets Pipeline'            # Set Pipeline as 'Individual Tickets'
deals['Deal Stage'] = 'Closed Won'                  # Set Deal Stage as 'Closed Won'
deals['Deal type'] = 'Individual Ticket'            # Set Deal type as 'Individual Ticket
deals['Season'] = Season                            # Set Season as the current season

# Assign Deal Owner based on Business Unit
deals['Deal Owner'] = deals['BusinessUnit'].map({
    'Baysox': 'info@baysox.com',
    'Keys': 'info@frederickkeys.com'
}).fillna('')

# Convert 'Event date' to string before concatenation
deals['Deal Name'] = deals['BusinessUnit'] + ' ' + deals['Event date'].astype(str) + ' ' + deals['TotalTickets'].astype(str) + ' Tickets'

# Map the BusinessUnit values and assign them back to the column
deals['BusinessUnit'] = deals['BusinessUnit'].map({
    'Baysox': 'Bowie Baysox',
    'Keys': 'Frederick Keys'
}).fillna('')

# Rename columns in the Deals dataframe
deals.rename(columns={
    'TransactionID': 'TicketReturn transaction number',
    'TotalTickets': 'Number of Tickets',
    'TotalPrice': 'Amount',
    'TicketType': 'Individual Ticket Type'
}, inplace=True)

# Separate rows where Email is known and where it is not
deals_known_email = deals[deals['Email'].notna()]
deals_unknown_email = deals[deals['Email'].isna()]

# Drop specified columns
deals_unknown_email = deals_unknown_email.drop(columns=['Email'])

# exported_deals each to a separate CSV file
deals_known_email.to_csv('2024.11.11 - Attain - Individual Baseball Tickets Email.csv', index=False)
deals_unknown_email.to_csv('2024.11.11 - Attain - Individual Baseball Tickets No Email.csv', index=False)


# Step 5 - Export Deals from HubSpot and Merge with Line Items

When updating deals in HubSpot you need the Record ID of the deal. Since line items will be added onto the deals we need to <b> export the deals we created along with their Record ID, TicketReturn transaction number, and Event Date </b> to merge the Record IDs for the Deals to the line items. 

In doing this we are using the <b> line_items </b> dataframe that we had saved previously after cleaning our data <br>
Export the deals from HubSpot and save them as <b> export.csv </b> in the same location as the code

1. <b> Match the Event Date columns </b> -- Since we are merging on [Event Date] we need to ensure that the formats of the event dates are the same across dataframes
2. <b> Merge data </b> -- Here we merge the data to get the Record ID of deals associated with the line items. We do a left join because we know that for each deal there must be associating line items as deals all have at least one ticket associated with them and the line items are representative of the tickets. However, it is import to <b> check all line items have a matching deal </b> before importing, therefore a warning is sent if that is not the case. This could be due to an incorrect export and should be checked.
3. <b> Create Line Item properties </b> -- There are 3 required properties for creating a line item in HubSpot. These 3 properties are Name, Quantity, and Price. To create these we start with a naming convention which is the 'Ticket Type + Section'. We follow this up with creating the Price and Quantity properties. These take all the tickets with the same name from the transaction and counts the number of tickets and takes the average price in order to have the amount in our line items match the amount of the deal.

Once these are merged and added we can export this file for import into HubSpot. HubSpot can't take line items with negative values so it is important to set line items with values less than zero to equal zero before importing (there should be none anyway)

To import into HubSpot:
1. Select Deals and Line Items
2. Set to update Deals and create Line Items
3. Select Record ID as a deal property matching to Record ID
4. Set Name, Quantity, and Price to be line item properties under the same names
5. This should result in no errors in the preview and can go ahead and import

In [12]:
exported_deals = pd.read_csv('export.csv')
line_items = pd.read_csv('line items.csv')

# Merge on 'Event Date' and 'TicketReturn transaction number' from exported_deals
merged_line_items = pd.merge(line_items, exported_deals[['Event Date', 'TicketReturn transaction number', 'Record ID']],
                     left_on=['Event date', 'TransactionID'], 
                     right_on=['Event Date', 'TicketReturn transaction number'],
                     how='left')  # Use 'left' join to keep all rows from tickets

# Check for any unmatched line items (where Record ID is NaN)
unmatched_line_items = merged_line_items[merged_line_items['Record ID'].isna()]

# Issue a warning if any unmatched line items are found
if not unmatched_line_items.empty:
    warnings.warn(f"{len(unmatched_line_items)} line items have no matching deal record. Find an associated deal before importing, check HubSpot on transaction number to ensure deal was originally imported.")
else:
    print('All lines items have been merged to a deal')

# Drop the 'TicketReturn transaction number' column as it's no longer needed after the merge
merged_line_items = merged_line_items.drop('TicketReturn transaction number', axis=1)

merged_line_items['Line Item Name'] = merged_line_items['Ticket Type'] + ' - ' + line_items['Section']

# Group by Record ID and Line Item Name
line_items_cleaned = merged_line_items.groupby(['Record ID', 'Line Item Name']).agg(
    Price=('PriceCorrected', 'mean'),  # Calculate the average of PriceCorrected as Price
    Quantity=('Line Item Name', 'count')  # Count the number of rows as Quantity
).reset_index()

# Set any negative Price values to 0 and format to two decimal places
line_items_cleaned['Price'] = line_items_cleaned['Price'].apply(lambda x: max(float(x), 0)).round(2)

line_items_cleaned.to_csv('2024.11.11 - Attain - Individual Baseball Line Items.csv')

C:\Users\MaxStevens\AppData\Local\Temp\ipykernel_16836\2564772614.py:2: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  line_items = pd.read_csv('line items.csv')


All lines items have been merged to a deal
